In [1]:
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr_torch_local.models import DeepFM
from deepctr_torch_local.inputs import SparseFeat, DenseFeat, get_feature_names

/mnt/data/jjw/anaconda3/envs/torch10/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-05-15 11:16:27.925376: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-05-15 11:16:28.145667: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-15 11:16:29.358803: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-15 11:16:29.380633: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performanc

In [23]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import re
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cab

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm

import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore')

In [24]:
data = pd.read_csv('./criteo_sample.txt')

In [25]:
data.shape 

(200, 40)

In [26]:
data.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,NaN,3,260.0,NaN,17668.0,NaN,NaN,33.0,NaN,...,e5ba7672,87c6f83c,NaN,NaN,0429f84b,NaN,3a171ecb,c0d61a5c,NaN,NaN
1,0,NaN,-1,19.0,35.0,30251.0,247.0,1.0,35.0,160.0,...,d4bb7bd8,6fc84bfb,NaN,NaN,5155d8a3,NaN,be7c41b4,ded4aac9,NaN,NaN
2,0,0.0,0,2.0,12.0,2013.0,164.0,6.0,35.0,523.0,...,e5ba7672,675c9258,NaN,NaN,2e01979f,NaN,bcdee96c,6d5d1302,NaN,NaN
3,0,NaN,13,1.0,4.0,16836.0,200.0,5.0,4.0,29.0,...,e5ba7672,52e44668,NaN,NaN,e587c466,NaN,32c7478e,3b183c5c,NaN,NaN
4,0,0.0,0,104.0,27.0,1990.0,142.0,4.0,32.0,37.0,...,e5ba7672,25c88e42,21ddcdc9,b1252a9d,0e8585d2,NaN,32c7478e,0d4a6d1a,001f3601,92c878de


In [27]:
sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

In [28]:
data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']

In [29]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

In [30]:
# TODO: 分桶

In [31]:
mms = MinMaxScaler(feature_range=(0,1))
data[dense_features] = mms.fit_transform(data[dense_features])

In [32]:
data.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,0,0.0,0.001332,0.092362,0.000000,0.034825,0.000000,0.000000,0.673469,0.000000,...,8,66,0,0,3,0,1,96,0,0
1,0,0.0,0.000000,0.006750,0.402299,0.059628,0.117284,0.003322,0.714286,0.154739,...,7,52,0,0,47,0,7,112,0,0
2,0,0.0,0.000333,0.000710,0.137931,0.003968,0.077873,0.019934,0.714286,0.505803,...,8,49,0,0,25,0,6,53,0,0
3,0,0.0,0.004664,0.000355,0.045977,0.033185,0.094967,0.016611,0.081633,0.028046,...,8,37,0,0,156,0,0,32,0,0
4,0,0.0,0.000333,0.036945,0.310345,0.003922,0.067426,0.013289,0.653061,0.035783,...,8,14,5,3,9,0,0,5,1,47


In [33]:
fixlen_feature_columns = \
    [SparseFeat(feat, vocabulary_size=data[feat].nunique(), embedding_dim=4) for i, feat in enumerate(sparse_features)] + \
    [DenseFeat(feat, 1,) for feat in dense_features]

In [34]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [35]:
seed = 2023

In [36]:
train, test = train_test_split(data, test_size=0.2)

In [37]:
train.shape, test.shape 

((160, 40), (40, 40))

In [38]:
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

In [39]:
device = 'cuda:2'

In [40]:
model = DeepFM(linear_feature_columns, dnn_feature_columns, task='binary', device=device)
model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'])

In [45]:
history = model.fit(train_model_input, train[target].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2, )

cuda:2
Train on 128 samples, validate on 32 samples, 1 steps per epoch
Epoch 1/10
0s - loss:  0.0690 - binary_crossentropy:  0.0690 - val_binary_crossentropy:  0.6900
Epoch 2/10
0s - loss:  0.0641 - binary_crossentropy:  0.0641 - val_binary_crossentropy:  0.6965
Epoch 3/10
0s - loss:  0.0584 - binary_crossentropy:  0.0584 - val_binary_crossentropy:  0.7048
Epoch 4/10
0s - loss:  0.0524 - binary_crossentropy:  0.0524 - val_binary_crossentropy:  0.7151
Epoch 5/10
0s - loss:  0.0463 - binary_crossentropy:  0.0463 - val_binary_crossentropy:  0.7273
Epoch 6/10
0s - loss:  0.0406 - binary_crossentropy:  0.0406 - val_binary_crossentropy:  0.7415
Epoch 7/10
0s - loss:  0.0353 - binary_crossentropy:  0.0353 - val_binary_crossentropy:  0.7579
Epoch 8/10
0s - loss:  0.0306 - binary_crossentropy:  0.0306 - val_binary_crossentropy:  0.7764
Epoch 9/10
0s - loss:  0.0266 - binary_crossentropy:  0.0266 - val_binary_crossentropy:  0.7971
Epoch 10/10
0s - loss:  0.0231 - binary_crossentropy:  0.0231 - v

In [27]:
pred_ans = model.predict(test_model_input, batch_size=256)

In [33]:
pred_ans = pred_ans.flatten()

In [34]:
pred_ans

array([0.44601488, 0.42801806, 0.42037854, 0.42484626, 0.43087134,
       0.42532471, 0.41749066, 0.40677869, 0.4108181 , 0.42355424,
       0.40990862, 0.40437853, 0.42273125, 0.43629241, 0.42149687,
       0.44408071, 0.41925356, 0.41810825, 0.40554896, 0.42757764,
       0.41953444, 0.41979948, 0.40969521, 0.41959932, 0.42349553,
       0.42517036, 0.42788035, 0.429198  , 0.42487895, 0.42382473,
       0.42748493, 0.41551584, 0.42312342, 0.42289767, 0.42630863,
       0.42289525, 0.41653109, 0.43582463, 0.42616656, 0.43875551])

In [35]:
test['label'].values 

array([0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

In [36]:
roc_auc_score(test['label'].values, pred_ans)

0.37666666666666665